In [69]:
# Delete specific folder in colab Files, if they're not empty
!rm -rf /content/Data

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [52]:
!unzip /content/gdrive/MyDrive/Fireimages.zip

Archive:  /content/gdrive/MyDrive/Fireimages.zip
  inflating: fire_dataset/fire_images/fire.1.png  
  inflating: fire_dataset/fire_images/fire.10.png  
  inflating: fire_dataset/fire_images/fire.100.png  
  inflating: fire_dataset/fire_images/fire.101.png  
  inflating: fire_dataset/fire_images/fire.102.png  
  inflating: fire_dataset/fire_images/fire.103.png  
  inflating: fire_dataset/fire_images/fire.104.png  
  inflating: fire_dataset/fire_images/fire.105.png  
  inflating: fire_dataset/fire_images/fire.106.png  
  inflating: fire_dataset/fire_images/fire.107.png  
  inflating: fire_dataset/fire_images/fire.108.png  
  inflating: fire_dataset/fire_images/fire.109.png  
  inflating: fire_dataset/fire_images/fire.11.png  
  inflating: fire_dataset/fire_images/fire.110.png  
  inflating: fire_dataset/fire_images/fire.111.png  
  inflating: fire_dataset/fire_images/fire.112.png  
  inflating: fire_dataset/fire_images/fire.113.png  
  inflating: fire_dataset/fire_images/fire.114.png  
 

In [4]:
!unzip /content/gdrive/MyDrive/CNN_CombinedDataset.zip

Archive:  /content/gdrive/MyDrive/CNN_CombinedDataset.zip
   creating: CCN_CombinedDataset/Testing/
   creating: CCN_CombinedDataset/Testing/fire/
  inflating: CCN_CombinedDataset/Testing/fire/abc001.jpg  
  inflating: CCN_CombinedDataset/Testing/fire/abc002.jpg  
  inflating: CCN_CombinedDataset/Testing/fire/abc003.jpg  
  inflating: CCN_CombinedDataset/Testing/fire/abc004.jpg  
  inflating: CCN_CombinedDataset/Testing/fire/abc005.jpg  
  inflating: CCN_CombinedDataset/Testing/fire/abc006.jpg  
  inflating: CCN_CombinedDataset/Testing/fire/abc007.jpg  
  inflating: CCN_CombinedDataset/Testing/fire/abc008.jpg  
  inflating: CCN_CombinedDataset/Testing/fire/abc009.jpg  
  inflating: CCN_CombinedDataset/Testing/fire/abc010.jpg  
  inflating: CCN_CombinedDataset/Testing/fire/abc011.jpg  
  inflating: CCN_CombinedDataset/Testing/fire/abc012.jpg  
  inflating: CCN_CombinedDataset/Testing/fire/abc013.jpg  
  inflating: CCN_CombinedDataset/Testing/fire/abc014.jpg  
  inflating: CCN_CombinedDa

In [5]:
#importing the libraries
import keras
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from keras.layers import Dropout
from keras.layers import BatchNormalization

In [11]:
classifier = Sequential() # Initialise the CNN

In [12]:
# Ist step of Convoltional layer to get feature maps using feature detector
classifier.add(Convolution2D(filters=32, # output feature maps
                             kernel_size=(3,3), # matrix size for feature detector
                             input_shape=(128, 128, 3), # input image shape, 3 is for rgb coloured image with 128*128 px
                             kernel_initializer='he_uniform', # weights distribution
                             activation='relu')) # activation function
# 2nd Pooling layer
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [13]:
# Adding another Convoltional layer
classifier.add(Convolution2D(filters=32, # output feature maps
                             kernel_size=(3,3), # matrix size for feature detector
                             kernel_initializer='he_uniform', # weights distribution
                             activation='relu')) # activation function
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [14]:
classifier.add(Convolution2D(filters=64,
                             kernel_size=(3,3), 
                             kernel_initializer='he_uniform', 
                             activation='relu'))
# 3rd Pooling layer
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [15]:
from tensorflow.keras.regularizers import l2
#convolutional and pooling layer.
classifier.add(Convolution2D(filters=64,
                             kernel_size=(3,3), 
                             kernel_initializer='he_uniform', 
                             activation='relu'))

In [16]:
# Step 3 - Flattening
classifier.add(Flatten())

In [17]:
#Step 4 full connection in which input we have from flattening
classifier.add(Dense(units=128,kernel_initializer='glorot_uniform', activation='relu'))
#step 5 output layer
classifier.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [18]:
from keras.preprocessing.image import ImageDataGenerator
#applying all the transformation we want to apply to training data set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
#Rescaling the test data set images to use for validation.
test_datagen= ImageDataGenerator(rescale=1./255)
#Getting My training data ready for validation, so it will read all the data with the px size we gave.
training_set= train_datagen.flow_from_directory(directory= '/content/CCN_CombinedDataset/Training and Validation',
                                               target_size=(128, 128), # As we choose 64*64 for our convolution model
                                               batch_size=64,
                                               class_mode='binary' # for 2 class binary 
                                               )


Found 2519 images belonging to 2 classes.


In [19]:
test_set= test_datagen.flow_from_directory(directory= '/content/CCN_CombinedDataset/Testing',
                                               target_size=(128,128), # As we choose 64*64 for our convolution model
                                               batch_size=64,
                                               class_mode='binary' # for 2 class binary
                                          )

Found 380 images belonging to 2 classes.


In [20]:
classifier.fit(training_set, #training data to fit
                        steps_per_epoch = 39, # Data in training set (training_size/batch_size= No of batches)
                        epochs=10, # No of epochs to run,  Iterations
                        validation_data = test_set, # Test or validation set 
                        validation_steps = 6               )

Epoch 1/10
39/39 [==============================] - 89s 2s/step - loss: 0.4729 - accuracy: 0.8399 - val_loss: 0.3385 - val_accuracy: 0.8974
Epoch 2/10
39/39 [==============================] - 88s 2s/step - loss: 0.1633 - accuracy: 0.9373 - val_loss: 0.3618 - val_accuracy: 0.8711
Epoch 3/10
39/39 [==============================] - 88s 2s/step - loss: 0.1486 - accuracy: 0.9503 - val_loss: 0.2642 - val_accuracy: 0.9263
Epoch 4/10
39/39 [==============================] - 87s 2s/step - loss: 0.1366 - accuracy: 0.9564 - val_loss: 0.2466 - val_accuracy: 0.9158
Epoch 5/10
39/39 [==============================] - 88s 2s/step - loss: 0.1096 - accuracy: 0.9609 - val_loss: 0.2265 - val_accuracy: 0.9184
Epoch 6/10
39/39 [==============================] - 88s 2s/step - loss: 0.0843 - accuracy: 0.9711 - val_loss: 0.2207 - val_accuracy: 0.9289
Epoch 7/10
39/39 [==============================] - 89s 2s/step - loss: 0.0912 - accuracy: 0.9707 - val_loss: 0.2100 - val_accuracy: 0.9395
Epoch 8/10
39/39 [==

In [21]:
score = classifier.evaluate(test_set, verbose = 0 )
print("Test Score: ", score[0])
print("Test accuracy: ", score[1])

Test Score:  0.17518681287765503
Test accuracy:  0.9578947424888611


In [22]:
classifier.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 61, 61, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 30, 30, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 14, 14, 64)      

In [23]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/content/testimage_fire.png', target_size = (128, 128))
# Loading the image and converting the pixels into array whcih will be used as input to predict.
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'no fire'
else:
    prediction = 'fire'
print(prediction)

fire


In [24]:
print(test_set.class_indices)

{'fire': 0, 'nofire': 1}
